In [1]:
import numpy as np
from copy import deepcopy
from collections import deque
from string import ascii_lowercase, ascii_uppercase
from sys import stdout

In [2]:
#Check if position is traversable
def good_pos(graph, pos):
    x = pos[0]
    y = pos[1]
    if x < 0 or x >= len(graph):
        return False
    if y < 0 or y >= len(graph[x]):
        return False
    value = graph[x][y]
    return value != '#'

#Breadth-First Search
def BFS(graph, start, end=None, get_path=False, get_dist=False, get_junction=False):
    dx = [-1, 1]
    dy = [-1, 1]
    queue = deque([start])
    dist = {start: 0}
    
    junction = []
    
    while len(queue):
        cur_pos = queue.popleft()
        cur_dist = dist[cur_pos]
        if cur_pos == end:
            break
        adj = 0
        for i in range(0, 2):
            nxt_dist = cur_dist + 1
            #move in x
            nxt_pos = (cur_pos[0]+dx[i], cur_pos[1])
            if good_pos(graph, nxt_pos) :
                if nxt_pos not in dist.keys():
                    queue.append(nxt_pos)
                    dist[nxt_pos] = nxt_dist
                adj += 1
            #move in y
            nxt_pos = (cur_pos[0], cur_pos[1]+dy[i])
            if good_pos(graph, nxt_pos):
                if nxt_pos not in dist.keys():
                    queue.append(nxt_pos)
                    dist[nxt_pos] = nxt_dist
                adj += 1
        if adj > 2 and cur_pos not in junction:
            junction.append(cur_pos)
            
    if get_junction:
        return junction
                
    max_dist = 0
    for key in dist.keys():
        if dist[key] > max_dist:
            max_dist = dist[key]
    
    if get_path and end is not None:
        path = [end]
        test_dist = max_dist
        dxy = [[1,0],[-1,0],[0,1],[0,-1]]
        while path[-1] != start:
            for delta in dxy:
                test = (path[-1][0]+delta[0],path[-1][1]+delta[1])
                if test in dist and dist[test] < test_dist:
                    path.append(test)
                    test_dist -= 1
                    break
        return path
    
    if get_dist:
        return dist, max_dist
    
    return max_dist

def collect_keys(paths):
    print('Solving...')
    #Care about where we have been, where we are and the distance traveled.
    #Don't care about the order of where we have been
    #Thus, if we have been to the same places and are at the same place, keep the shortest
    stdout.write('0')

    routes = {}
    dist = 0
    possible = paths['@']
    can_get_to = []
    for key in possible.keys():
        if len(possible[key][1]) == 0:
            can_get_to.append([key, possible[key][0]])

    for j in range(0, len(can_get_to)):
        key = can_get_to[j][0]
        extra = can_get_to[j][1]
        tmp_route = (key, key)
        if tmp_route in routes.keys():
            if routes[tmp_route] > dist+extra:
                routes[tmp_route] = dist+extra
        else:
            routes[tmp_route] = dist+extra

    for i in range(1, len(paths['@'].keys())):
        stdout.write('.'+str(i))
        new_routes = {}
        for route in routes.keys():
            last = route[1]
            dist = routes[route]
            possible = paths[last]

            can_get_to = []
            for key in possible.keys():
                if key in route[0]:
                    continue
                have_got = True
                for door in possible[key][1]:
                    if door not in route[0]:
                        have_got = False
                if have_got:
                    can_get_to.append([key, possible[key][0]])

            for j in range(0, len(can_get_to)):
                key = can_get_to[j][0]
                extra = can_get_to[j][1]
                tmp_route = (''.join(sorted(route[0]+key)), key)
                if tmp_route in new_routes.keys():
                    if new_routes[tmp_route] > dist+extra:
                        new_routes[tmp_route] = dist+extra
                else:
                    new_routes[tmp_route] = dist+extra

        routes = deepcopy(new_routes)

    stdout.write('\n')
    stdout.flush()
    shortest = -1
    for key in routes.keys():
        if shortest == -1 or routes[key] < shortest:
            shortest = routes[key]
            
    return shortest

In [3]:
data = np.genfromtxt('day18_input.txt', comments=None, delimiter='\n', dtype=np.str)

maze = []
for i in range(0, len(data)):
    maze.append(list(data[i]))
maze = np.array(maze)

start = np.where(maze == '@')
start = (start[0][0], start[1][0])

#Find names and positions of keys
key_pos = {}
for letter in ascii_lowercase:
    ltr = np.where(maze == letter)
    try:
        x = ltr[0][0]
    except:
        continue
    key_pos[letter] = (ltr[0][0], ltr[1][0])
#print(key_pos)

#Find names and positions of doors
door_pos = {}
for letter in ascii_uppercase:
    ltr = np.where(maze == letter)
    try:
        x = ltr[0][0]
    except:
        continue
    door_pos[letter] = (ltr[0][0], ltr[1][0])
#print(door_pos)

#Get paths between keys including needed keys
paths = {}
single_path = {}
#print(start)
for key in key_pos:
    pth = BFS(maze, start, end=key_pos[key], get_path=True)
    dist = len(pth)-1
    blok = []
    for KEY in door_pos:
        if door_pos[KEY] in pth:
            blok.append(KEY.lower())
    single_path[key] = [dist, blok]
paths['@'] = single_path

for key in key_pos:
    single_path = {}
    for key2 in key_pos:
        if key == key2:
            continue
        pth = BFS(maze, key_pos[key], end=key_pos[key2], get_path=True)
        dist = len(pth)-1
        blok = []
        for KEY in door_pos:
            if door_pos[KEY] in pth:
                blok.append(KEY.lower())
        single_path[key2] = [dist, blok]
    paths[key] = single_path
#print(paths['@'])

In [4]:
shortest = collect_keys(paths)
print('Part 1 Solution:', shortest)

Solving...
0.1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.25
Part 1 Solution: 4520


In [5]:
#Edit maze for part 2
start = np.where(maze == '@')
start = (start[0][0], start[1][0])

maze[start[0]-1, start[1]-1] = '@'
maze[start[0]-1, start[1]+1] = '@'
maze[start[0]+1, start[1]-1] = '@'
maze[start[0]+1, start[1]+1] = '@'

maze[start[0]+1, start[1]] = '#'
maze[start[0]-1, start[1]] = '#'
maze[start[0],   start[1]] = '#'
maze[start[0], start[1]+1] = '#'
maze[start[0], start[1]-1] = '#'

#Split maze into four mazes and solve each
mazes = []
mazes.append(maze[0:start[0]+1,0:start[1]+1])
mazes.append(maze[0:start[0]+1,start[1]:])
mazes.append(maze[start[0]:,0:start[1]+1])
mazes.append(maze[start[0]:,start[1]:])
mazes = np.array(mazes)

In [6]:
shortest = 0
for maze in mazes:
    start = np.where(maze == '@')
    start = (start[0][0], start[1][0])
    
    #Find names and positions of keys
    key_pos = {}
    for letter in ascii_lowercase:
        ltr = np.where(maze == letter)
        try:
            x = ltr[0][0]
        except:
            continue
        key_pos[letter] = (ltr[0][0], ltr[1][0])
    #print(key_pos)
    
    #Find names and positions of doors
    #Only for doors whos keys are within the maze
    door_pos = {}
    for letter in key_pos.keys():
        ltr = np.where(maze == letter.upper())
        try:
            x = ltr[0][0]
        except:
            continue
        door_pos[letter] = (ltr[0][0], ltr[1][0])
    #print(door_pos)
    
    #Get paths between keys including needed keys
    paths = {}
    single_path = {}
    #print(start)
    for key in key_pos:
        pth = BFS(maze, start, end=key_pos[key], get_path=True)
        dist = len(pth)-1
        blok = []
        for KEY in door_pos:
            if door_pos[KEY] in pth:
                blok.append(KEY.lower())
        single_path[key] = [dist, blok]
    paths['@'] = single_path
    
    for key in key_pos:
        single_path = {}
        for key2 in key_pos:
            if key == key2:
                continue
            pth = BFS(maze, key_pos[key], end=key_pos[key2], get_path=True)
            dist = len(pth)-1
            blok = []
            for KEY in door_pos:
                if door_pos[KEY] in pth:
                    blok.append(KEY.lower())
            single_path[key2] = [dist, blok]
        paths[key] = single_path
        
    #print(paths['@'])
    shortest += collect_keys(paths)
    
print('Part 2 Solution:', shortest)

Solving...
0.1.2.3.4.5
Solving...
0.1
Solving...
0.1.2.3.4.5.6.7.8
Solving...
0.1.2.3.4.5.6.7.8
Part 2 Solution: 1540
